<a href="https://colab.research.google.com/github/annanasnas/askqe/blob/main/baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from huggingface_hub import login
login()

In [2]:
!git clone https://github.com/askqe/askqe.git

fatal: destination path 'askqe' already exists and is not an empty directory.


In [5]:
import torch
import json
from transformers import pipeline
from tqdm.notebook import tqdm
import os
import sys


if torch.cuda.is_available():
    torch.cuda.empty_cache()

pipe = pipeline(
    "text-generation",
    model="google/gemma-2-2b-it",
    model_kwargs={"dtype": torch.bfloat16},
    device="cuda", # Используем GPU
)
pipe.tokenizer.padding_side = "left"

BATCH_SIZE = 32

atomic_fact_prompt = """Task: You will be given an English sentence. Your goal is to identify a list of atomic facts from the sentence. Atomic fact is a short sentence conveying one piece of information. Output the list of atomic facts in Python list format without giving any additional explanation. Do not output as code format (```python```).

*** Example Starts ***
Sentence: The number of accessory proteins and their function is unique depending on the specific coronavirus.
Atomic facts: ['The number of accessory proteins is unique depending on the specific coronavirus.', 'The function of accessory proteins is unique depending on the specific coronavirus.']
*** Example Ends ***

Sentence: {{sentence}}
Atomic facts: """

input_file = f"/content/askqe/biomqm/dev_with_backtranslation.jsonl"
output_file = f"/content/askqe/baseline/askqe_atomic_facts_backup.jsonl"


prompts = []
data_buffer = []


print("Prompts initialization")
with open(input_file, "r", encoding="utf-8") as f:
    for line in f:
        try:
            d = json.loads(line)
            if "src" in d:
                data_buffer.append(d)
                p = atomic_fact_prompt.replace("{{sentence}}", d["src"])

                full_p = f"<start_of_turn>user\n{p}<end_of_turn>\n<start_of_turn>model\n"
                prompts.append(full_p)
        except json.JSONDecodeError:
            continue

print("Pipe initialization")
results_iterator = pipe(
    prompts,
    batch_size=BATCH_SIZE,
    max_new_tokens=512,
    return_full_text=False,
)

print("Starting of fact generation")
final_results = []
for out in tqdm(results_iterator, total=len(prompts), desc="Fact generation..."):
    generated_text = out[0]["generated_text"].strip()
    if generated_text.endswith("<end_of_turn>"):
        generated_text = generated_text[:-len("<end_of_turn>")].strip()

    final_results.append(generated_text)

with open(output_file, "w", encoding="utf-8") as f:
    for d, res in zip(data_buffer, final_results):
        d["atomic_facts"] = res
        f.write(json.dumps(d, ensure_ascii=False) + "\n")

Запуск через Transformers Pipeline (Plan B)...


model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda


✅ Pipeline успешно загружен.
Чтение данных и подготовка промптов...
🚀 Запуск генерации для 5216 примеров (Batch Size: 32)...


KeyboardInterrupt: 